In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import random

In [2]:
# Create m x n matrix for Y
Y_train = np.loadtxt('./data/train.txt').astype(int)
Y_test = np.loadtxt('./data/test.txt').astype(int)

def center(train, test):
    avg = np.mean(train[:,2])
    train[:,2] = train[:,2 ] - avg
    test[:,2] = test[:, 2] - avg

center(Y_train, Y_test)

### Method 2 (Adding Bias Term a and b)

In [3]:
def grad_U(Ui, Yij, Vj, reg, eta, a, b):
    return eta * (np.dot(reg, Ui) - (np.dot(Vj, ((Yij) - np.dot(Ui, Vj) - a - b ))))

def grad_V(Ui, Yij, Vj, reg, eta, a, b):
    return eta * (np.dot(reg, Vj) - (np.dot(Ui, ((Yij) - np.dot(Ui, Vj) - a - b ))))

def grad_A(Ui, Yij, Vj, reg, eta, a, b):
    return -eta * ((Yij) - np.dot(Ui, Vj) - a - b  - (reg * a))
    
def grad_B(Ui, Yij, Vj, reg, eta, a, b):
    return -eta * ((Yij) - np.dot(Ui, Vj) - a - b  - (reg * b))
    
def get_err(U, V, Y, A, B, reg=0.0):
    err = 0.0
    for (i, j, Yij) in Y:
        err += .5 * (((Yij) - (np.dot(U[i-1], V[j-1]) + A[i-1] + B[j-1])) ** 2)
        
    err += 0.5 * reg * (np.linalg.norm(U)**2 + np.linalg.norm(V)**2 
                    + np.linalg.norm(A)**2 + np.linalg.norm(B)**2)
    
    return err / len(Y)


In [7]:
def train_model(M, N, K, eta, reg, Y, eps=0.0001, max_epochs=300):
  
    itr = 0
    U = np.random.uniform(-0.5, 0.5, (M, K))
    V = np.random.uniform(-0.5, 0.5, (N, K))

    A = np.random.uniform(-0.5, 0.5, (M))
    B = np.random.uniform(-0.5, 0.5, (N))
   
    curr_loss = get_err(U, V, Y, A, B)
    
    shuffled = np.random.permutation(list(range(len(Y))))
    
    for k in range(len(Y)):
        i, j, Yij = Y[shuffled[k]]
        
        # update U, V, A, B
        u = grad_U(U[i-1], Yij, V[j-1], reg, eta, A[i-1], B[j-1])
        v = grad_V(U[i-1], Yij, V[j-1], reg, eta, A[i-1], B[j-1])
        a = grad_A(U[i-1], Yij, V[j-1], reg, eta, A[i-1], B[j-1])
        b = grad_B(U[i-1], Yij, V[j-1], reg, eta, A[i-1], B[j-1])
        
        U[i-1] = U[i-1] - u 
        V[j-1] = V[j-1] - v
        A[i-1] = A[i-1] - a
        B[j-1] = B[j-1] - b
        
    next_loss = get_err(U, V, Y, A, B)
    init = curr_loss - next_loss
    curr_delta = init
    curr_loss = next_loss
    
    while itr < max_epochs and (curr_delta / init) > eps:
        shuffled = np.random.permutation(list(range(len(Y))))
        for k in range(len(Y)): 
            i, j, Yij = Y[shuffled[k]]
            
            # update U, V, A, B
            u = grad_U(U[i-1], Yij, V[j-1], reg, eta, A[i-1], B[j-1])
            v = grad_V(U[i-1], Yij, V[j-1], reg, eta, A[i-1], B[j-1])
            a = grad_A(U[i-1], Yij, V[j-1], reg, eta, A[i-1], B[j-1])
            b = grad_B(U[i-1], Yij, V[j-1], reg, eta, A[i-1], B[j-1])

            U[i-1] = U[i-1] - u 
            V[j-1] = V[j-1] - v
            A[i-1] = A[i-1] - a
            B[j-1] = B[j-1] - b
            
        next_loss = get_err(U,V,Y, A,B)
        
        curr_delta = curr_loss - next_loss
        curr_loss = next_loss
        itr += 1
        
    return U, V, curr_loss, A, B

In [9]:
# Create 943 users x 1682 movies 
M = max(max(Y_train[:,0]), max(Y_test[:,0])).astype(int) # users
N = max(max(Y_train[:,1]), max(Y_test[:,1])).astype(int) # movies
print("Factorizing with ", M, " users, ", N, " movies.")

k = 20
reg = 0.0
eta = 0.03 # learning rate

U, V, E_in, A, B = train_model(M, N, k, eta, reg, Y_train)
E_out = get_err(U, V, Y_test, A, B)

print("Training error is: " + str(E_in))
print("Test error is: " + str(E_out))

Factorizing with  943  users,  1682  movies.
0.06657169014459653 63
Training error is: 0.06657169014459653
Test error is: 0.3510576991240275


In [ ]:
test_reg Y_test [10**-3, 10**-2, 10**-1, 1, 10]

for r in test_reg:
    U_reg, V_reg, E_in_reg, A_reg, B_reg = train_model(M, N, k, eta, r, Y_train)
    E_out_test = get_err(U_reg, V_reg, Y_test, A, B)
    

### New heading